# 画像の分類

Congnitive Servicesの**Computer Vision**では、画像を扱うための便利なモデルがあらかじめ用意されていますが、コンピュータビジョンのモデルを自分でトレーニングする必要がある場合も多いでしょう。例えば、小売業のNorthwind Traders社が、チェックアウト時にカメラで撮影した画像に基づいて、顧客が購入したい食料品を識別する自動チェックアウトシステムを作りたいとします。そのためには、画像を分類して購入する商品を識別できる分類モデルを学習する必要があります。
 
 ![A robot holding a clipboard, classifying pictures of an apple, a banana, and an orange](./images/image-classification.jpg)

Azureでは、***Custom Vision*** Cognitive Serviceを使って、既存の画像に基づいて画像分類モデルをトレーニングすることができます。画像分類ソリューションの作成には2つの要素があります。まず、既存の画像を使って異なるクラスを認識するモデルをトレーニングします。モデルが学習されたら、それをアプリケーションが利用できるサービスとして公開します。

## Custom Vision リソースを作成する

Custom Visionサービスを使用するには、モデルの**トレーニング**に使用するAzureリソースと、アプリケーションが使用できるようにモデルを**公開**するためのリソースが必要です。どちらか（または両方）のタスクのためのリソースは、一般的な**Cognitive Services**リソース、または特定の**Custom Vision**リソースになります。これらのタスクのそれぞれに同じCognitive Servicesリソースを使用することができますが、コストを別々に管理したい場合は、各タスクに異なるリソース（同じリージョン内）を使用することもできます。

以下の手順で、新しい**Custom Vision**リソースを作成してください。

1. 新しいブラウザタブを開き、[https://portal.azure.com](https://portal.azure.com)のAzureポータルを開き、Azureサブスクリプションに関連付けられたMicrosoftアカウントを使用してサインインします。
2. **&#65291;リソースの作成**ボタンを選択し、*custom vision*を検索して、以下の設定で**Custom Vision**リソースを作成します。
    - **作成オプション**: 両方
    - **サブスクリプション**: *ご自身のサプスクリプション*
    - **リソースグループ**: *既存のリソースグループを選択するか、ユニークな名前で新しいリソースグループを作成します。*
    - **名前**: *ユニークな名前を入力*
    - **リージョン**: *利用可能なリージョンを選択（例:東日本）*
    - **トレーニング価格レベル**: F0
    - **予測価格レベル**: F0

    > **補足**: サブスクリプションにF0のカスタムビジョンサービスがすでにある場合、このサービスには**S0**を選択してください。

4. リソースが作成されるのを待っていると、トレーニング用と予測用の2つのCustom Visionリソースがプロビジョニングされていることに気がつきます。リソースを作成したリソースグループに移動すると、これらを確認することができます。

## Custom Visionプロジェクトを作成する

オブジェクト検出モデルをトレーニングするには、トレーニングリソースに基づいてCustom Visionプロジェクトを作成する必要があります。そのためには、Custom Visionポータルを使用します。

1. https://aka.ms/fruit-images からトレーニングイメージが入った圧縮ファイルをダウンロードし、それをローカルコンピューター上で解凍します（**注意**: トレーニングイメージにアクセスできない場合は、一時的な回避策として https://www.github.com にアクセスしたあと、https://aka.ms/fruit-images にアクセスしてみてください。）
2. 別のブラウザタブで、[https://customvision.ai](https://customvision.ai)のCustom Visionポータルを開きます。プロンプトが表示されたら、Azureサブスクリプションに関連付けられたMicrosoftアカウントを使用してサインインし、利用規約に同意します。
3. Custom Visionポータルで、以下の設定で新しいプロジェクトを作成します。
    - **Name**: Grocery Checkout
    - **Description**: Image classification for groceries
    - **Resource**: *前のステップで作成したCustom Visionリソース*
    - **Project Types**: Classification
    - **Classification Types**: Multiclass (single tag per image)
    - **Domains**: Food
4. **Add images**ボタンをクリックして、前のステップで解凍して取り出した**apple**フォルダーの中の全ての画像ファイルを選択しアップロードします。アップロードできたら、次のように*apple*というタグをつけます。
    
    ![Upload apple with apple tag](./images/upload_apples.jpg)
   
5. 前述の手順を繰り返して、**banana**フォルダ内の画像には*banana*というタグを、**orange**フォルダ内の画像には*orange*というタグを付けてアップロードします。
6. Custom Vision プロジェクトにアップロードした画像を見てみましょう。このように、各クラス15枚の画像があるはずです。
   
   ![Tagged images of fruit - 15 apples, 15 bananas, and 15 oranges](./images/fruit.jpg)
    
7. Custom Vision プロジェクトで、画像の上にある「**Train**」をクリックして、タグ付けされた画像を使って分類モデルをトレーニングします。**Quick Training**オプションを選択し、トレーニングの反復が完了するのを待ちます（これには1分ほどかかることがあります）。
8.  モデルの反復学習が完了したら、**Precision(適合率)**、**Recall(再現率)**、**AP(平均適合率)** のパフォーマンスメトリクスを確認してください - これらは分類モデルの予測精度を測定し、すべて高い値を示しています。

## モデルをテストする

この反復モデルをアプリケーションに公開する前に、テストを行う必要があります。

1. パフォーマンス指標の上にある「**Quick Test**」ボタンをクリックします。
2. **Image URL** ボックスに`https://aka.ms/apple-image`と入力し、 &#10132; をクリックします。
3. モデルから返された予測を見ると、次のように*apple*の確率スコアが最も高くなっているはずです。  
    
    ![An image with a class prediction of apple](./images/test-apple.jpg)

4. **Quick Test** ウィンドウを閉じます。

## 画像分類モデルの公開と公開後の利用

これで、学習したモデルを公開し、クライアントアプリケーションから利用する準備が整いました。

1. **&#128504; Publish** をクリックして、以下の設定でトレーニングされたモデルを公開（パブリッシュ）します。
    - **Model name**: groceries
    - **Prediction Resource**: *以前に作成した予測用のリソース*.

### (!) チェックイン 
モデル名が正しく**groceries**と入力されていることを確認してください。


2.  公開したら、**Performance**ページの右上にある**設定**（&#9881;）アイコンをクリックして、プロジェクトの設定を表示します。次に、**General**（左）の下にある**Project Id**をコピーします。スクロールダウンして、ステップ13の下のコードセルの **YOUR_PROJECT_ID** の部分にペーストして置き換えます。

    ![Project ID in project settings](./images/cv_project_settings.jpg)

> _**補足**: この演習の最初に **Custom Vision** リソースを作成する代わりに **Cognitive Services** リソースを使用した場合、プロジェクト設定の右側からそのキーとエンドポイントをコピーして、下のコードセルに貼り付け、実行して結果を確認することができます。そうでなければ、以下のステップを続けて、Custom Vision予測リソースのキーとエンドポイントを取得します。_

3. **Project Settings**ページの左上にある**Projects Gallery** (&#128065;)アイコンをクリックすると、Custom Visionポータルのホーム画面に戻り、プロジェクトが表示されます。

4. Custom Vision ポータルのホーム・ページの右上にある**設定** (&#9881;)アイコンをクリックして、Custom Vision Serviceの設定を表示します。次に、**Resources**の下で、**prediction**リソース（<u>トレーニング用リソースではありません</u>）を展開し、その**Key**と**Endpoint**の値をステップ5の下のコードセルにコピーし、**YOUR_KEY**と**YOUR_ENDPOINT**を置き換えます。

### (!) チェックイン
**Custom Vision**リソースを使用している場合、**prediction**リソース（<u>トレーニング用リソースではありません</u>）を使用しましたか？

![Prediction resource key and endpoint in custom vision settings](./images/cv_settings.jpg)

5. セルの左側にある **Run Cell** (&#9655;）ボタンをクリックして、以下のコードセルを実行し、変数にプロジェクトID、キー、エンドポイントの値を設定します。

In [12]:
project_id = 'YOUR_PROJECT_ID'
cv_key = 'YOUR_KEY'
cv_endpoint = 'YOUR_ENDPOINT'
model_name = 'groceries' # this must match the model name you set when publishing your model iteration (it's case-sensitive)!
print('Ready to predict using model {} in project {}'.format(model_name, project_id))

Ready to predict using model groceries in project YOUR_PROJECT_ID


これで、Custom Visionクライアントでキーとエンドポイントを使用して、Custom Vision分類モデルに接続できるようになります。

次のコードセルを実行して，公開したモデルを使ってテスト画像を分類します．

> **補足**: コードの詳細についてはあまり気にしないでください。このコードはComputer Vision SDK for Pythonを使って、/data/image-classification/test-fruitフォルダにある各画像のクラス予測を取得しています。

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Get the test images from the data/vision/test folder
test_folder = os.path.join('data', 'image-classification', 'test-fruit')
test_images = os.listdir(test_folder)

# Create an instance of the prediction service
credentials = ApiKeyCredentials(in_headers={"Prediction-key": cv_key})
custom_vision_client = CustomVisionPredictionClient(endpoint=cv_endpoint, credentials=credentials)

# Create a figure to display the results
fig = plt.figure(figsize=(16, 8))

# Get the images and show the predicted classes for each one
print('Classifying images in {} ...'.format(test_folder))
for i in range(len(test_images)):
    # Open the image, and use the custom vision model to classify it
    image_contents = open(os.path.join(test_folder, test_images[i]), "rb")
    classification = custom_vision_client.classify_image(project_id, model_name, image_contents.read())
    # The results include a prediction for each tag, in descending order of probability - get the first one
    prediction = classification.predictions[0].tag_name
    # Display the image with its predicted class
    img = Image.open(os.path.join(test_folder, test_images[i]))
    a=fig.add_subplot(len(test_images)/3, 3,i+1)
    a.axis('off')
    imgplot = plt.imshow(img)
    a.set_title(prediction)
plt.show()

あなたが作成した画像分類モデルは、画像の中の食料品を正しく識別してくれていることでしょう。

## さらに学ぶ

Custom Visionサービスには、今回の演習で紹介した以外にも様々な機能があります。例えば、Custom Visionサービスを使用して、画像内のオブジェクトを分類するだけでなく、画像内のオブジェクトの位置を示す*バウンディングボックス*を識別する**オブジェクト検出**モデルを作成することもできます。

Custom Vision cognitive service についてもっと学びたい場合は、[Custom Vision とは](https://docs.microsoft.com/azure/cognitive-services/custom-vision-service/home)を参照してください。